# Project 2

Alex Hindelang and Florian Michael-Schwarzinger

# Q: What is the relationship between genetically modified T cells (CAR-T) and cancerous leukemia cells?

An experimental new "living drug" to fight cancer is being put to use: genetically modified T cells. The cells are altered with a chimeric antigen receptor protein (denoted by the CAR acronym), which has the ability to target specific cells and is often derived from viruses like HIV. This is one of the most promising therapies for cancer treatment, but it comes with some potential dangers. The proliferation of CAR-T cells in the body can lead to inflammation and neurological toxicity. As such, it is important to monitor both the CAR-T cell growth and the cancerous cell growth.

We wish to investigate the difference between two strategies for reducing CAR-T toxicity: suicide genes, and activation molecules (ON-switch).

In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

We will first use a function to define the system values.

In [2]:
def make_system(Tbirth, Tdeath, cancerbirth, cancerdeath):
    """Make a system object of different rates    
    returns: System object
    """
    init = State(carT = 1, cancer = 1) #This is extremely not the value we're going with

    t0 = 0
    t_end = 200 #this is also arbitrary

    return System(init=init, t0=t0, t_end=t_end, Tbirth = Tbirth, Tdeath = Tdeath, 
                  cancerbirth = cancerbirth, cancerdeath = cancerdeath)

In [1]:
def plot_results(x, y):
    """Plot the results of a Lotka-Volterra model
    
    x: TimeSeries
    y: TimeSeries
    """
    plot(x, '--', label='Prey')
    plot(y, '-', label='Predator')
    decorate(xlabel='Time (days)',
             ylabel='Fraction of Population')